In [18]:
import numpy as np
import pandas as pd
# bring in cc data
df_cc = pd.read_csv(r'UH60 Corrosion JAN 2000 to MAR 2019 (unfiltered).csv', encoding ='latin1', low_memory=False)
#df_cc = pd.read_csv(r'C:\Users\John.Case\OneDrive - West Point\Backup\ORCEN\CPO\UH60 Corrosion JAN 2000 to MAR 2019 (unfiltered).csv', encoding ='latin1', low_memory=False)

In [19]:
# Set 'ID' as the index of the dataframe
df_cc = df_cc.reset_index()
# convert to datetime
df_cc['date1_2410'] = pd.to_datetime(df_cc['DATE_2410'], errors='coerce')
# Subset cc data to 2017
df_cc_subdates = df_cc[(df_cc['date1_2410']>=np.datetime64('2017-01-01T00:00:00.000000000')) & (df_cc['date1_2410']<=np.datetime64('2017-12-31T23:59:59.000000000'))]

In [20]:
df_cc_subdates.columns

Index(['index', 'COPY', 'DATE_2410', 'PN_NOUN', 'NSN', 'PN', 'SN',
       'DESCRIPTION', 'REMARKS', 'SYS_CAT', 'NHA_SN', 'UIC', 'CITY',
       'ORG_NAME', 'QTY', 'YEAR', 'NOUN', 'PRICE', 'date1_2410'],
      dtype='object')

In [21]:
# This operation requires you to build a delayed pipeline of pandas DataFrame manipulations. 
# You will then convert the output to a Dask DataFrame in which each file will be one chunk.
# Define @delayed-function 
import dask.dataframe as dd
from dask.delayed import delayed
import pandasql as ps

@delayed
def filter_join_chunks(chunk, df_cc_subdates):

    # Read in the DataFrame: df
    df = pd.DataFrame(chunk)
    # subset to only UH60s
    df = df[df['TMS'].str.contains("UH", na=False)]
    # convert dates to datetime and subset to 2017
    df['maint_start_date'] = pd.to_datetime(df['MaintStartDate'], errors='coerce')
    df['maint_end_date'] = pd.to_datetime(df['MaintCompletionDate'], errors='coerce')
    df = df[(df['maint_start_date']>=np.datetime64('2017-01-01T00:00:00.000000000')) & (df['maint_end_date']<=np.datetime64('2017-12-31T23:59:59.000000000'))]
    
    ### left join onto the CC df by UIC, dates, and parts tree OBJs
    
    # join on UIC == UnitCD, then on dates (I should be able to do this without SQL...)
    sqlcode = '''
    select *
    from df_cc_subdates
    inner join df where df_cc_subdates.UIC == df.UnitCD and df_cc_subdates.date1_2410 >= df.maint_start_date and df_cc_subdates.date1_2410 <= df.maint_end_date
    '''

    df = ps.sqldf(sqlcode,locals())
    
    #need to drop all text after 'NOT' for each Tier1 and Tier2 object
    sep = 'NOT'
    tier1_obj = []
    for text in df.Tier1Object.values:
        tier1_obj.append(text.split(sep, 1)[0])
    tier2_obj = []
    for text in df.Tier2Object.values:
        tier2_obj.append(text.split(sep, 1)[0])
    # append columns to the df
    df['tier1_obj'] = tier1_obj
    df['tier2_obj'] = tier2_obj
    
    df = df.astype('object')
    # Return df
    return df

# Loop over chunks of the MADW csv
list_of_dfs = []
for chunk in pd.read_csv(r"/Users/mh302/Desktop/corrected_full.csv", chunksize=10**5, dtype=object):  # 25MB chunks
    # r"/Users/mh302/Desktop/corrected.csv"
    # Apply read_flights to filename; append to dataframes
    list_of_dfs.append(filter_join_chunks(chunk, df_cc_subdates = df_cc_subdates))

# This creates a dask df to parallelize over given a bunch of pandas dfs

cols = ['index', 'COPY', 'DATE_2410', 'PN_NOUN', 'NSN', 'PN', 'SN',
       'DESCRIPTION', 'REMARKS', 'SYS_CAT', 'NHA_SN', 'UIC', 'CITY',
       'ORG_NAME', 'QTY', 'YEAR', 'NOUN', 'PRICE', 'date1_2410', 'AvailCD',
       'CalcLaborCorroCost', 'CalcLaborCost', 'CalcMatlCorroCost',
       'CalcMatlCost', 'CorrectiveCorroCost', 'CorrectiveMaintCost',
       'CorroPerc', 'DataSource', 'EndItemUniqueID', 'EngineTMS',
       'InstallationMasterName', 'ESIZone', 'LMIWBS', 'LMIWBSNIIN',
       'MaintActionCD', 'MaintActionFY', 'MaintCompletionDate', 'MaintDLH',
       'MaintOperationDetailDesc', 'MaintStartDate', 'Maint_TAT',
       'OPNComplDate', 'OPNStartDate', 'OrigLaborCost', 'orig_maint_tat',
       'OrigMatlCost', 'PartCorroCost', 'PartMaintCost',
       'PreventiveCorrective', 'PreventiveCorroCost', 'PreventiveMaintCost',
       'SorCD', 'StructureCorroCost', 'StructureMaintCost', 'TMS',
       'TotalCalcCorroCost', 'TotalCalcCost', 'UnitCD', 'Tier1Object',
       'Tier2Object', 'maint_start_date', 'maint_end_date', 'tier1_obj',
       'tier2_obj']
meta = pd.DataFrame(index=[], data=[], columns=cols, dtype='object')
joined_cc_madw = dd.from_delayed(list_of_dfs, meta)
# meta = pd.DataFrame(index=i[:0], data=[], columns=gts, dtype=np.float64)
# Print some computed value to make the delayed tasks actually run
#print(len(joined_cc_madw['NOUN']).compute())

In [22]:
len(joined_cc_madw.NOUN.compute())

1224273

In [24]:
joined_cc_madw.compute().to_csv(r'/Users/mh302/Desktop/2017_joined_cc_madw.csv', encoding='utf-8', index=False)

## All below is not run on MADW to cc joins

In [1]:
import dask.dataframe as dd
dd.read_csv(r'D:\CPOdata\DoD_REF24.csv', blocksize=25e6)  # 25MB chunks  )

In [1]:

chunksize = 10 ** 5
filename = r'D:\CPOdata\DoD_REF24.csv'
num_chunks = 200
n=0
list_of_dfs = []
for chunk in pd.read_csv(filename, chunksize=chunksize, encoding ='latin1', low_memory=False):
    df = pd.DataFrame(chunk)
    # subset to only UH60s
    df = df[df['TMS'].str.contains("UH")]
    # convert dates to datetime and subset to 2017
    df['maint_start_date'] = pd.to_datetime(df['MaintStartDate'], errors='coerce')
    df['maint_end_date'] = pd.to_datetime(df['MaintCompletionDate'], errors='coerce')
    df = df[(df['maint_start_date']>=np.datetime64('2017-01-01T00:00:00.000000000')) & (df['maint_end_date']<=np.datetime64('2017-12-31T23:59:59.000000000'))]
    list_of_dfs.append(df)
    n+=1
    if n % 10 == 0:
        print(n)
    if n == num_chunks:
        break
df = pd.concat(list_of_dfs, axis=0)

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200


In [ ]:
from joblib import Parallel, delayed
import multiprocessing

import pandasql as ps

sqlcode = '''
select *
from df_cc_subdates
inner join df where df_cc_subdates.date1_2410 >= df.maint_start_date and df_cc_subdates.date1_2410 <= df.maint_end_date
'''

joined_dates_df = ps.sqldf(sqlcode,locals())


def join_dfs(df_chunk, df_cc= df_cc, sqlcode = sqlcode):
    list_of_rows = []
    for word in doc.split():
        if word.lower() not in stp_wds:
            filtered_words.append(word.lower())
        else:
            next
    return(' '.join(filtered_words)) 

num_cores = multiprocessing.cpu_count()
chunk_size = len(df) // num_cores
x_no_stp_wds = Parallel(n_jobs=num_cores)(delayed(remove_stp_wds)(doc) for df_chunk in df)

In [22]:
import numpy as np
import pandas as pds

df = pds.DataFrame(np.random.rand(14,4), columns=['a', 'b', 'c', 'd'])

def chunker(seq, size):
    return (seq[pos:pos + size] for pos in xrange(0, len(seq), size))

for i in chunker(df,5):
    print i

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(i)? (<ipython-input-22-e44cba6888b9>, line 10)

In [2]:
df.columns

Index(['AvailCD', 'CalcLaborCorroCost', 'CalcLaborCost', 'CalcMatlCorroCost',
       'CalcMatlCost', 'CorrectiveCorroCost', 'CorrectiveMaintCost',
       'CorroPerc', 'DataSource', 'EndItemUniqueID', 'EngineTMS',
       'InstallationMasterName', 'ESIZone', 'LMIWBS', 'LMIWBSNIIN',
       'MaintActionCD', 'MaintActionFY', 'MaintCompletionDate', 'MaintDLH',
       'MaintOperationDetailDesc', 'MaintStartDate', 'Maint_TAT',
       'OPNComplDate', 'OPNStartDate', 'OrigLaborCost', 'orig_maint_tat',
       'OrigMatlCost', 'PartCorroCost', 'PartMaintCost',
       'PreventiveCorrective', 'PreventiveCorroCost', 'PreventiveMaintCost',
       'SorCD', 'StructureCorroCost', 'StructureMaintCost', 'TMS',
       'TotalCalcCorroCost', 'TotalCalcCost', 'UnitCD', 'Tier1Object',
       'Tier2Object', 'maint_start_date', 'maint_end_date'],
      dtype='object')

In [3]:
set(df['TMS'])

{'EUH-60L', 'UH-60A', 'UH-60L', 'UH-60M', 'UH-60N', 'UH-60Q'}

In [4]:
len(df)

434283

In [5]:
df.iloc[0:5, 0:10]

,AvailCD,CalcLaborCorroCost,CalcLaborCost,CalcMatlCorroCost,CalcMatlCost,CorrectiveCorroCost,CorrectiveMaintCost,CorroPerc,DataSource,EndItemUniqueID
1280011,(null),0.4695,1.1738,0.4301,1.0753,0.0,0.0000,0.4,TEMP17,(null)
1281481,(null),0.0000,7.0438,0.0000,30.9019,0.0,37.9457,0.0,TEMP17,(null)
1281483,(null),0.0000,7.1148,0.0000,1.0753,0.0,8.1901,0.0,TEMP17,(null)
1281486,(null),0.0000,7.1148,0.0000,3.5059,0.0,0.0000,0.0,TEMP17,(null)
1281487,(null),0.0000,7.1148,0.0000,3.5059,0.0,0.0000,0.0,TEMP17,(null)


In [6]:
df.iloc[0:5, 10:20]

,EngineTMS,InstallationMasterName,ESIZone,LMIWBS,LMIWBSNIIN,MaintActionCD,MaintActionFY,MaintCompletionDate,MaintDLH,MaintOperationDetailDesc
1280011,(null),Frc Jacksonville,14,RI117,(null),(null),FY17,2017-08-31 00:00:00,0.33,PERFORM CARD 5-005 TASK
1281481,(null),Frc/Mcas Cherry Point,15,RD071,013527887,(null),FY17,2017-08-31 00:00:00,1.98,DISASSEMBLY
1281483,(null),Frc Jacksonville,14,RS025,(null),(null),FY17,2017-08-31 00:00:00,2,TFC AS MARKED
1281486,(null),FRC North Island,13,RO064,(null),(null),FY17,2017-04-02 06:00:00,2,MAG ETCH
1281487,(null),FRC North Island,13,RO064,(null),(null),FY17,2017-04-02 06:00:00,2,MAG ETCH


In [7]:
df.iloc[0:5, 20:30]

,MaintStartDate,Maint_TAT,OPNComplDate,OPNStartDate,OrigLaborCost,orig_maint_tat,OrigMatlCost,PartCorroCost,PartMaintCost,PreventiveCorrective
1280011,2017-04-01 00:00:00,(null),2017-04-11 00:00:00,2017-04-11 00:00:00,19.8162,(null),6.7821,0.8996,2.2491,PREVENTIVE
1281481,2017-04-18 00:00:00,(null),2017-08-03 00:00:00,2017-08-02 00:00:00,118.897,(null),194.9529,0.0000,37.9457,CORRECTIVE
1281483,2017-04-01 00:00:00,(null),2017-04-13 00:00:00,2017-04-11 00:00:00,120.098,(null),6.7821,0.0000,0.0000,CORRECTIVE
1281486,2017-04-01 00:00:00,(null),2017-04-01 06:00:00,2017-04-01 00:00:00,120.098,(null),22.1150,0.0000,10.6207,NEITHER
1281487,2017-04-01 00:00:00,(null),2017-04-01 06:00:00,2017-04-01 00:00:00,120.098,(null),22.1150,0.0000,10.6207,NEITHER


In [8]:
df.iloc[0:5, 30:]

,PreventiveCorroCost,PreventiveMaintCost,SorCD,StructureCorroCost,StructureMaintCost,TMS,TotalCalcCorroCost,TotalCalcCost,UnitCD,Tier1Object,Tier2Object,maint_start_date,maint_end_date
1280011,0.8996,2.2491,FRCSE,0.0,0.0000,UH-60A,0.8996,2.2491,FRCSE,NAVIGATIONAL CONTROL HARDWARE,TAIL CONE,2017-04-01,2017-08-31 00:00:00
1281481,0.0000,0.0000,FRCE,0.0,0.0000,UH-60A,0.0000,37.9457,FRCE,(null),PRESSURE REGULATOR,2017-04-18,2017-08-31 00:00:00
1281483,0.0000,0.0000,FRCSE,0.0,8.1901,UH-60N,0.0000,8.1901,FRCSE,(null),DOOR,2017-04-01,2017-08-31 00:00:00
1281486,0.0000,0.0000,FRCSW,0.0,0.0000,UH-60N,0.0000,10.6207,FRCSW,GEARBOX,GEAR ASSEMBLY,2017-04-01,2017-04-02 06:00:00
1281487,0.0000,0.0000,FRCSW,0.0,0.0000,UH-60A,0.0000,10.6207,FRCSW,GEARBOX,GEAR ASSEMBLY,2017-04-01,2017-04-02 06:00:00


In [9]:
df['MaintOperationDetailDesc'].values[0:5]

array(['PERFORM CARD 5-005 TASK', 'DISASSEMBLY', 'TFC AS MARKED',
       'MAG ETCH', 'MAG ETCH'], dtype=object)

In [10]:
df['MaintStartDate'].dtype
df['maint_start_date'] = pd.to_datetime(df['MaintStartDate'], errors='coerce')
df['maint_end_date'] = pd.to_datetime(df['MaintCompletionDate'], errors='coerce')

In [11]:
df['maint_start_date'].values[0]

numpy.datetime64('2015-10-06T00:00:00.000000000')

In [12]:
import numpy as np
time_diff = df['maint_start_date'].values[1] - df['maint_start_date'].values[0]
# convert the timedelta to days
time_diff.astype('timedelta64[D]') / np.timedelta64(1, 'D')

386.0

In [10]:
print('Max date:', max(df['maint_start_date'].values))
print('Min date:', min(df['maint_start_date'].values))

Max date: 2017-12-31T00:00:00.000000000
Min date: 2017-01-04T00:00:00.000000000


In [14]:
#import dateutil.parser as dparser
#dparser.parse("monkey 10/01/1980 love banana",fuzzy=True, dayfirst=True)

In [11]:
# bring in cc data
df_cc = pd.read_csv(r'C:\Users\John.Case\OneDrive - West Point\Backup\ORCEN\CPO\UH60 Corrosion JAN 2000 to MAR 2019 (unfiltered).csv', encoding ='latin1', low_memory=False)

In [12]:
# Set 'ID' as the index of the dataframe
df_cc = df_cc.reset_index()

In [13]:
#len(df_cc)
df_cc

,index,COPY,DATE_2410,PN_NOUN,NSN,PN,SN,DESCRIPTION,REMARKS,SYS_CAT,NHA_SN,UIC,CITY,ORG_NAME,QTY,YEAR,NOUN,PRICE
0,0,1,4-Jan-00,4046T28G05/ANTI-ICE VALVE,3.00E+12,4046T28G05,COM05302,070-Broken,NaN,T700,GEE207752,WBM2AA,FT BENNING SOUTH,498TH MED CO,1,2000,ANTI-ICE VALVE,"$5,594.00"
1,1,1,7-Jan-00,3046T17G01/O/S DRAIN VALVE ASSY,2.92E+12,3046T17G01,APM935HK,520-Pitted,NaN,T700,GEE375377,W0H93H,FT CAMPBELL NORTH,PROJECT OLR,1,2000,O/S DRAIN VALVE ASSY,"$3,376.00"
2,2,1,10-Jan-00,6039T58G02/COMBUSTION LINER,2.84E+12,6039T58G02,GGMU8031,070-Broken,NaN,T700,GEE306791,W0U431,FT CAMPBELL NORTH,DYNCORP ENHANCED ENG REP ACTY (ESRA),1,2000,COMBUSTION LINER,"$15,793.00"
3,3,1,10-Jan-00,6043T89G01/POWER TURBINE MODULE,2.84E+12,6043T89G01,GEL037174,070-Broken,NaN,T700,GEE207831,K07ER5,EL PASO,DOS AVIATION INC,1,2000,POWER TURBINE MODULE,"$119,464.00"
4,4,1,11-Jan-00,6055T88G06/STAGE 1 NOZZLE ASSY,2.84E+12,6055T88G06,AMDA5164,170-Corroded,NaN,T700,GEE375146,WYDGA0,JACKSONVILLE,A CO 1ST/111TH,1,2000,STAGE 1 NOZZLE ASSY,"$11,390.00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44567,44567,1,6-Mar-19,70150-09100-043/MAIN ROTOR BLADE ASSY,1.62E+12,70150-09100-043,A00712774,170-Corroded,NaN,UH60,9526663,W8NBAA,FRANKFORT,KYARNG BOONE NG CENTER,1,2019,MAIN ROTOR BLADE ASSY,"$140,812.00"
44568,44568,1,6-Mar-19,"70200-27000-049/BOX ASSY, CENTER STABILA",1.56E+12,70200-27000-049,A21610327,170-Corroded,NaN,UH60,1020277,WC08A0,FT BRAGG,A CO 2ND/82ND AVN,1,2019,"BOX ASSY, CENTER STABILA","$40,705.71"
44569,44569,3,6-Mar-19,70357-06300-042/INTERMEDIATE GEARBOX,1.68E+12,70357-06300-042,A00501248,070-Broken,"AS PER GREEN TAG, ASSET WAS RECEIVED IN THE FI...",UH60,NaN,W0MUAA,CORPUS CHRISTI,CORPUS CHRISTI ARMY DEPOT,1,2019,INTERMEDIATE GEARBOX,"$36,494.00"
44570,44570,1,6-Mar-19,"70400-08163-043/SUPPORT, BELL CRANK",3.04E+12,70400-08163-043,36505929,170-Corroded,NaN,UH60,9526612,WHPPA0,APO,A CO 2ND/2ND AVN,1,2019,"SUPPORT, BELL CRANK","$3,270.00"


In [14]:
df_cc['DATE_2410'].dtype
df_cc['date1_2410'] = pd.to_datetime(df_cc['DATE_2410'], errors='coerce')

In [15]:
print('Max date:', max(df_cc['date1_2410'].values))
print('Min date:', min(df_cc['date1_2410'].values))

Max date: 2019-03-07T00:00:00.000000000
Min date: 2000-01-01T00:00:00.000000000


In [20]:
x = np.datetime64('2016-01-01T00:00:00.000000000')
y = np.datetime64('2016-12-31T23:59:59.000000000')
(y-x).astype('timedelta64[D]')

numpy.timedelta64(365,'D')

In [16]:
# subset df_cc to the dates covered by the MADW df
df_cc_subdates = df_cc[(df_cc['date1_2410']>=min(df['maint_start_date'].values)) & (df_cc['date1_2410']<=max(df['maint_end_date'].values))]

In [17]:
df_cc_subdates

,index,COPY,DATE_2410,PN_NOUN,NSN,PN,SN,DESCRIPTION,REMARKS,SYS_CAT,NHA_SN,UIC,CITY,ORG_NAME,QTY,YEAR,NOUN,PRICE,date1_2410
39850,39850,3,7-Jan-17,6064T36P01/NO 6 BALL BEARING,3.11E+12,6064T36P01,TMKAD3AA,520-Pitted,NaN,T700,NaN,W0MUAA,CORPUS CHRISTI,CORPUS CHRISTI ARMY DEPOT,1,2017,NO 6 BALL BEARING,$561.00,2017-01-07
39851,39851,3,7-Jan-17,6064T36P01/NO 6 BALL BEARING,3.11E+12,6064T36P01,TMKAHV77,520-Pitted,NaN,T700,NaN,W0MUAA,CORPUS CHRISTI,CORPUS CHRISTI ARMY DEPOT,1,2017,NO 6 BALL BEARING,$561.00,2017-01-07
39852,39852,3,9-Jan-17,6043T45P01/NO 3 BALL BEARING,3.11E+12,6043T45P01,BARB0077,520-Pitted,NaN,T700,NaN,W0MUAA,CORPUS CHRISTI,CORPUS CHRISTI ARMY DEPOT,1,2017,NO 3 BALL BEARING,"$1,215.85",2017-01-09
39853,39853,3,9-Jan-17,6061T53P01/COMPRESSOR DISCHARGE SE,2.84E+12,6061T53P01,MLEFJ781,170-Corroded,NaN,T700,NaN,W0H96Y,CORPUS CHRISTI,AMCOM SAFR,1,2017,COMPRESSOR DISCHARGE SE,"$1,779.00",2017-01-09
39854,39854,3,9-Jan-17,6064T15P03/NO 4 ROLLER BEARING,3.11E+12,6064T15P03,FCAE6223,520-Pitted,NaN,T700,NaN,W0MUAA,CORPUS CHRISTI,CORPUS CHRISTI ARMY DEPOT,1,2017,NO 4 ROLLER BEARING,$946.00,2017-01-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41826,41826,1,27-Dec-17,"70103-08108-041/BLADE PIN, SOLID",5.32E+12,70103-08108-041,B94100287,117-Deteriorated,NaN,UH60,627089,WHQAA0,APO,A CO 2ND/3RD GSAB,1,2017,"BLADE PIN, SOLID",$960.16,2017-12-27
41827,41827,1,27-Dec-17,"70103-08108-041/BLADE PIN, SOLID",5.32E+12,70103-08108-041,B94100106,240-Flaking,NaN,UH60,627089,WHQAA0,APO,A CO 2ND/3RD GSAB,1,2017,"BLADE PIN, SOLID",$960.16,2017-12-27
41828,41828,1,27-Dec-17,70150-09100-043/MAIN ROTOR BLADE ASSY,1.62E+12,70150-09100-043,A00716213,070-Broken,CRACKED ON LOWER CORNER OF ROOT STA. 42,UH60,8624490,WPFED0,FT LEWIS,D CO 1ST/168TH,1,2017,MAIN ROTOR BLADE ASSY,"$140,812.00",2017-12-27
41829,41829,3,28-Dec-17,70400-08102-045/AFT BELLCRANK,1.56E+12,70400-08102-045,C34502641,520-Pitted,NaN,UH60,NaN,W0H95C,FT CAMPBELL,OLR FT CAMPBELL,1,2017,AFT BELLCRANK,"$1,937.00",2017-12-28


In [23]:
# subset dfs for testing just to 2017
df_cc_subdates = df_cc[(df_cc['date1_2410']>=np.datetime64('2017-01-01T00:00:00.000000000')) & (df_cc['date1_2410']<=np.datetime64('2017-12-31T23:59:59.000000000'))]
df = df[(df['maint_start_date']>=np.datetime64('2017-01-01T00:00:00.000000000')) & (df['maint_end_date']<=np.datetime64('2017-12-31T23:59:59.000000000'))]

In [18]:
print('Length of cc df: ', len(df_cc_subdates))
print('Length of MADW df: ', len(df))

Length of cc df:  1967
Length of MADW df:  434283


In [19]:
import pandasql as ps

sqlcode = '''
select *
from df_cc_subdates
inner join df where df_cc_subdates.date1_2410 >= df.maint_start_date and df_cc_subdates.date1_2410 <= df.maint_end_date
'''

uic_dates_df = ps.sqldf(sqlcode,locals())

In [20]:
uic_dates_df

,index,COPY,DATE_2410,PN_NOUN,NSN,PN,SN,DESCRIPTION,REMARKS,SYS_CAT,...,StructureCorroCost,StructureMaintCost,TMS,TotalCalcCorroCost,TotalCalcCost,UnitCD,Tier1Object,Tier2Object,maint_start_date,maint_end_date
0,39850,3,7-Jan-17,6064T36P01/NO 6 BALL BEARING,3.11E+12,6064T36P01,TMKAD3AA,520-Pitted,None,T700,...,24.1688,60.4221,UH-60A,24.1688,60.4221,FRCSE,(null),BODY FRAME OR HULL,2017-01-04 00:00:00.000000,2017-01-20 09:00:00.000000
1,39850,3,7-Jan-17,6064T36P01/NO 6 BALL BEARING,3.11E+12,6064T36P01,TMKAD3AA,520-Pitted,None,T700,...,0.0000,0.0000,UH-60N,0.0000,14.1776,FRCSE,(null),DOCUMENTS,2017-01-04 00:00:00.000000,2017-01-20 09:00:00.000000
2,39850,3,7-Jan-17,6064T36P01/NO 6 BALL BEARING,3.11E+12,6064T36P01,TMKAD3AA,520-Pitted,None,T700,...,0.0000,0.0000,UH-60A,0.0000,14.1776,FRCSE,(null),DOCUMENTS,2017-01-04 00:00:00.000000,2017-01-20 09:00:00.000000
3,39850,3,7-Jan-17,6064T36P01/NO 6 BALL BEARING,3.11E+12,6064T36P01,TMKAD3AA,520-Pitted,None,T700,...,24.1688,60.4221,UH-60N,24.1688,60.4221,FRCSE,(null),BODY FRAME OR HULL,2017-01-04 00:00:00.000000,2017-01-20 09:00:00.000000
4,39851,3,7-Jan-17,6064T36P01/NO 6 BALL BEARING,3.11E+12,6064T36P01,TMKAHV77,520-Pitted,None,T700,...,24.1688,60.4221,UH-60A,24.1688,60.4221,FRCSE,(null),BODY FRAME OR HULL,2017-01-04 00:00:00.000000,2017-01-20 09:00:00.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46205233,41830,3,29-Dec-17,70104-08010-044/STATIONARY SWASHPLATE,1.62E+12,70104-08010-044,151521527,520-Pitted,PITTED IN AREA K,UH60,...,0.0000,0.0000,UH-60M,11.3306,34.3353,WAYCA0,HELICOPTER DRIVE SYSTEM,TAIL ROTOR,2017-11-20 00:00:00.000000,2017-12-29 00:00:00.000000
46205234,41830,3,29-Dec-17,70104-08010-044/STATIONARY SWASHPLATE,1.62E+12,70104-08010-044,151521527,520-Pitted,PITTED IN AREA K,UH60,...,0.0000,0.0000,UH-60M,11.3306,34.3353,WP7TC0,(null),SHAFT,2017-12-28 00:00:00.000000,2017-12-29 00:00:00.000000
46205235,41830,3,29-Dec-17,70104-08010-044/STATIONARY SWASHPLATE,1.62E+12,70104-08010-044,151521527,520-Pitted,PITTED IN AREA K,UH60,...,0.0000,0.0000,UH-60M,11.3306,34.3353,WUAUA0,(null),BOX,2017-12-29 00:00:00.000000,2017-12-29 00:00:00.000000
46205236,41830,3,29-Dec-17,70104-08010-044/STATIONARY SWASHPLATE,1.62E+12,70104-08010-044,151521527,520-Pitted,PITTED IN AREA K,UH60,...,0.0000,0.0000,UH-60L,0.6569,6.5688,WV7PD0,ENGINE OR PROPULSION,FILLER CAP,2017-12-29 00:00:00.000000,2017-12-29 00:00:00.000000


In [21]:
uic_dates_df.to_csv(r'D:\CPOdata\2017_joined_cc_MADW_dates_df.csv', encoding='utf-8', index=False)

In [23]:
#need to drop all text after 'NOT' for each Tier1 and Tier2 object

sep = 'NOT'

tier1_obj = []

for text in uic_dates_df.Tier1Object.values:
    tier1_obj.append(text.split(sep, 1)[0])

tier2_obj = []

for text in uic_dates_df.Tier2Object.values:
    tier2_obj.append(text.split(sep, 1)[0])
    

In [24]:
# append columns to the df
uic_dates_df['tier1_obj'] = tier1_obj
uic_dates_df['tier2_obj'] = tier2_obj

In [25]:
import re

# did the date joining: dataframe is uic_dates_df
# now need to re-write this with just looking at finding NOUN in the tier1 and tier 2 for ITS OWN ROW.  should be MUCH QUICKER.

# this finds any word in NOUN within any tier1 or tier2 object
tier_1_match = []
tier_2_match = []
i=-1
for record in uic_dates_df['NOUN'].values:
    i += 1
    #print(record.split())
    if i % 100000 == 0:
        print(i)
    
    try:
        # check for NOUN in Tier1Object
        if re.compile('|'.join(record.split()),re.IGNORECASE).search(uic_dates_df['tier1_obj'].values[i]):
            tier_1_match.append(1)
        else:
            tier_1_match.append(0)
    except:
        tier_1_match.append(0)
        print("An exception (Tier1) occurred at index: ", i, uic_dates_df['NOUN'].values[i])
        
    try:       
        # check for NOUN in Tier2Object
        if re.compile('|'.join(record.split()),re.IGNORECASE).search(uic_dates_df['tier2_obj'].values[i]):
            tier_2_match.append(1)
        else:
            tier_2_match.append(0)
    except:
        tier_2_match.append(0)
        print("An exception (Tier2) occurred at index: ", i, uic_dates_df['NOUN'].values[i])

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000
5300000
5400000
5500000
5600000
5700000
5800000
5900000
6000000
6100000
6200000
6300000
6400000
6500000
6600000
6700000
6800000
6900000
7000000
7100000
7200000
7300000
7400000
7500000
7600000
7700000
7800000
7900000
8000000
8100000
8200000
8300000
8400000
8500000
8600000
8700000
8800000
8900000
9000000
9100000
9200000
9300000
9400000
9500000
9600000
9700000
9800000
9900000
10000000
10100000
10200000
10300000
10400000
10500000
10600000
10700000
10800000
10900000
11000000
11100000
11200000
11300000
11400000
11500000
11600000
11700000
11800000
11900000
12000000
12100000
12200000
12300000

In [26]:
uic_dates_df['tier_1_match'] = tier_1_match
uic_dates_df['tier_2_match'] = tier_2_match

In [27]:
# retain the unique cc id's
uic_dates_df['cc_id'] = uic_dates_df['index']
# new index column to track each joined record
uic_dates_df = uic_dates_df.reset_index()
uic_dates_df

,level_0,index,COPY,DATE_2410,PN_NOUN,NSN,PN,SN,DESCRIPTION,REMARKS,...,UnitCD,Tier1Object,Tier2Object,maint_start_date,maint_end_date,tier1_obj,tier2_obj,tier_1_match,tier_2_match,cc_id
0,0,39850,3,7-Jan-17,6064T36P01/NO 6 BALL BEARING,3.11E+12,6064T36P01,TMKAD3AA,520-Pitted,None,...,FRCSE,(null),BODY FRAME OR HULL,2017-01-04 00:00:00.000000,2017-01-20 09:00:00.000000,(null),BODY FRAME OR HULL,0,0,39850
1,1,39850,3,7-Jan-17,6064T36P01/NO 6 BALL BEARING,3.11E+12,6064T36P01,TMKAD3AA,520-Pitted,None,...,FRCSE,(null),DOCUMENTS,2017-01-04 00:00:00.000000,2017-01-20 09:00:00.000000,(null),DOCUMENTS,0,0,39850
2,2,39850,3,7-Jan-17,6064T36P01/NO 6 BALL BEARING,3.11E+12,6064T36P01,TMKAD3AA,520-Pitted,None,...,FRCSE,(null),DOCUMENTS,2017-01-04 00:00:00.000000,2017-01-20 09:00:00.000000,(null),DOCUMENTS,0,0,39850
3,3,39850,3,7-Jan-17,6064T36P01/NO 6 BALL BEARING,3.11E+12,6064T36P01,TMKAD3AA,520-Pitted,None,...,FRCSE,(null),BODY FRAME OR HULL,2017-01-04 00:00:00.000000,2017-01-20 09:00:00.000000,(null),BODY FRAME OR HULL,0,0,39850
4,4,39851,3,7-Jan-17,6064T36P01/NO 6 BALL BEARING,3.11E+12,6064T36P01,TMKAHV77,520-Pitted,None,...,FRCSE,(null),BODY FRAME OR HULL,2017-01-04 00:00:00.000000,2017-01-20 09:00:00.000000,(null),BODY FRAME OR HULL,0,0,39851
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46205233,46205233,41830,3,29-Dec-17,70104-08010-044/STATIONARY SWASHPLATE,1.62E+12,70104-08010-044,151521527,520-Pitted,PITTED IN AREA K,...,WAYCA0,HELICOPTER DRIVE SYSTEM,TAIL ROTOR,2017-11-20 00:00:00.000000,2017-12-29 00:00:00.000000,HELICOPTER DRIVE SYSTEM,TAIL ROTOR,0,0,41830
46205234,46205234,41830,3,29-Dec-17,70104-08010-044/STATIONARY SWASHPLATE,1.62E+12,70104-08010-044,151521527,520-Pitted,PITTED IN AREA K,...,WP7TC0,(null),SHAFT,2017-12-28 00:00:00.000000,2017-12-29 00:00:00.000000,(null),SHAFT,0,0,41830
46205235,46205235,41830,3,29-Dec-17,70104-08010-044/STATIONARY SWASHPLATE,1.62E+12,70104-08010-044,151521527,520-Pitted,PITTED IN AREA K,...,WUAUA0,(null),BOX,2017-12-29 00:00:00.000000,2017-12-29 00:00:00.000000,(null),BOX,0,0,41830
46205236,46205236,41830,3,29-Dec-17,70104-08010-044/STATIONARY SWASHPLATE,1.62E+12,70104-08010-044,151521527,520-Pitted,PITTED IN AREA K,...,WV7PD0,ENGINE OR PROPULSION,FILLER CAP,2017-12-29 00:00:00.000000,2017-12-29 00:00:00.000000,ENGINE OR PROPULSION,FILLER CAP,0,0,41830


In [28]:
# populate columns to id matches for objects and corrosion tag
obj_match_sum = []
corr_match = []

for i in range(0, len(uic_dates_df['index'])):
    obj_match_sum.append(uic_dates_df['tier_1_match'][i] + uic_dates_df['tier_2_match'][i])
    if uic_dates_df['CorroPerc'][i] != 0 and uic_dates_df['DESCRIPTION'][i] in ['170-Corroded', '520-Pitted', '050-Blistered', '240-Flaking']:
        corr_match.append(1)
    else:
        corr_match.append(0)
    if i % 10000 == 0:
        print(i)    
uic_dates_df['obj_match_sum'] = obj_match_sum
uic_dates_df['corr_match'] = corr_match

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000
1200000
1210000
1220000
1230000
1240000
1250000
1260000
1270000
1280000
1290000
1300000
1310000
1320000
1330000
1340000
1350000
1360000
1370000
1380000
13

10350000
10360000
10370000
10380000
10390000
10400000
10410000
10420000
10430000
10440000
10450000
10460000
10470000
10480000
10490000
10500000
10510000
10520000
10530000
10540000
10550000
10560000
10570000
10580000
10590000
10600000
10610000
10620000
10630000
10640000
10650000
10660000
10670000
10680000
10690000
10700000
10710000
10720000
10730000
10740000
10750000
10760000
10770000
10780000
10790000
10800000
10810000
10820000
10830000
10840000
10850000
10860000
10870000
10880000
10890000
10900000
10910000
10920000
10930000
10940000
10950000
10960000
10970000
10980000
10990000
11000000
11010000
11020000
11030000
11040000
11050000
11060000
11070000
11080000
11090000
11100000
11110000
11120000
11130000
11140000
11150000
11160000
11170000
11180000
11190000
11200000
11210000
11220000
11230000
11240000
11250000
11260000
11270000
11280000
11290000
11300000
11310000
11320000
11330000
11340000
11350000
11360000
11370000
11380000
11390000
11400000
11410000
11420000
11430000
11440000
11450000
1

19460000
19470000
19480000
19490000
19500000
19510000
19520000
19530000
19540000
19550000
19560000
19570000
19580000
19590000
19600000
19610000
19620000
19630000
19640000
19650000
19660000
19670000
19680000
19690000
19700000
19710000
19720000
19730000
19740000
19750000
19760000
19770000
19780000
19790000
19800000
19810000
19820000
19830000
19840000
19850000
19860000
19870000
19880000
19890000
19900000
19910000
19920000
19930000
19940000
19950000
19960000
19970000
19980000
19990000
20000000
20010000
20020000
20030000
20040000
20050000
20060000
20070000
20080000
20090000
20100000
20110000
20120000
20130000
20140000
20150000
20160000
20170000
20180000
20190000
20200000
20210000
20220000
20230000
20240000
20250000
20260000
20270000
20280000
20290000
20300000
20310000
20320000
20330000
20340000
20350000
20360000
20370000
20380000
20390000
20400000
20410000
20420000
20430000
20440000
20450000
20460000
20470000
20480000
20490000
20500000
20510000
20520000
20530000
20540000
20550000
20560000
2

28570000
28580000
28590000
28600000
28610000
28620000
28630000
28640000
28650000
28660000
28670000
28680000
28690000
28700000
28710000
28720000
28730000
28740000
28750000
28760000
28770000
28780000
28790000
28800000
28810000
28820000
28830000
28840000
28850000
28860000
28870000
28880000
28890000
28900000
28910000
28920000
28930000
28940000
28950000
28960000
28970000
28980000
28990000
29000000
29010000
29020000
29030000
29040000
29050000
29060000
29070000
29080000
29090000
29100000
29110000
29120000
29130000
29140000
29150000
29160000
29170000
29180000
29190000
29200000
29210000
29220000
29230000
29240000
29250000
29260000
29270000
29280000
29290000
29300000
29310000
29320000
29330000
29340000
29350000
29360000
29370000
29380000
29390000
29400000
29410000
29420000
29430000
29440000
29450000
29460000
29470000
29480000
29490000
29500000
29510000
29520000
29530000
29540000
29550000
29560000
29570000
29580000
29590000
29600000
29610000
29620000
29630000
29640000
29650000
29660000
29670000
2

37680000
37690000
37700000
37710000
37720000
37730000
37740000
37750000
37760000
37770000
37780000
37790000
37800000
37810000
37820000
37830000
37840000
37850000
37860000
37870000
37880000
37890000
37900000
37910000
37920000
37930000
37940000
37950000
37960000
37970000
37980000
37990000
38000000
38010000
38020000
38030000
38040000
38050000
38060000
38070000
38080000
38090000
38100000
38110000
38120000
38130000
38140000
38150000
38160000
38170000
38180000
38190000
38200000
38210000
38220000
38230000
38240000
38250000
38260000
38270000
38280000
38290000
38300000
38310000
38320000
38330000
38340000
38350000
38360000
38370000
38380000
38390000
38400000
38410000
38420000
38430000
38440000
38450000
38460000
38470000
38480000
38490000
38500000
38510000
38520000
38530000
38540000
38550000
38560000
38570000
38580000
38590000
38600000
38610000
38620000
38630000
38640000
38650000
38660000
38670000
38680000
38690000
38700000
38710000
38720000
38730000
38740000
38750000
38760000
38770000
38780000
3

In [29]:
# including multiple matches per cc record
# how many records have a tier1 and tier2 match?
#len(uic_dates_df[uic_dates_df['obj_match_sum'] == 2])

#how many records have a corr match?
#len(uic_dates_df[uic_dates_df['corr_match'] == 1])

# how many records match both tier1&2 and corr match?  
#len(uic_dates_df[(uic_dates_df['obj_match_sum'] == 2) & (uic_dates_df['corr_match'] == 1)] )

311338

In [30]:
uic_dates_df[(uic_dates_df['obj_match_sum']==1) & (uic_dates_df['corr_match']==0)]['level_0'].values[0]

4510

In [31]:
# for each unique corpus christi record, find the best match of objects and corr tag to keep
obj2x_corr_id = []
obj1x_corr_id = []
obj1x_id = []
no_match_ids = []

for id in set(uic_dates_df['index'].values):
    # set up emtpy df
    temp = pd.DataFrame()

    # subset to the joined records for each unique cc record
    temp = uic_dates_df[uic_dates_df['index'] == id]
    
    if not temp[(temp['obj_match_sum']==2) & (temp['corr_match']==1)].empty:
        # if exists a match of both tier 1 and tier 2 objs and corr tag match: get the id (level_0) for the record
         obj2x_corr_id.append(temp[(temp['obj_match_sum']==2) & (temp['corr_match']==1)]['level_0'].values[0])
        
    elif not temp[(temp['obj_match_sum']==1) & (temp['corr_match']==1)].empty:
        # if exists a match of either tier 1 or tier 2 objs and corr tag match: get the id (level_0) for the record
        obj1x_corr_id.append(temp[(temp['obj_match_sum']==1) & (temp['corr_match']==1)]['level_0'].values[0])
        
    elif not temp[(temp['obj_match_sum']==1) & (temp['corr_match']==0)].empty:
        # if exists a match of either tier 1 or tier 2 objs BUT NO corr tag match: get the id (level_0) for the record
        obj1x_id.append(temp[(temp['obj_match_sum']==1) & (temp['corr_match']==0)]['level_0'].values[0])
    else:
        # no obj match
        no_match_ids.append(id)

In [32]:
print('Records with 2x obj match and corr match:', len(obj2x_corr_id))
print('Proportion of records with 2x obj match and corr match:', len(obj2x_corr_id)/len(df_cc_subdates), '\n')
print('Records with 1x obj match and corr match:', len(obj1x_corr_id))
print('Proportion of records with 1x obj match and corr match:', len(obj1x_corr_id)/len(df_cc_subdates), '\n')
print('Records with 1x obj match but no corr match:', len(obj1x_id))
print('Proportion of records with 1x obj match but no corr match:', len(obj1x_id)/len(df_cc_subdates), '\n')
print('Records with no obj match:', len(no_match_ids))
print('Proportion of records with no obj match:', len(no_match_ids)/len(df_cc_subdates), '\n')

Records with 2x obj match and corr match: 352
Proportion of records with 2x obj match and corr match: 0.17895271987798678 

Records with 1x obj match and corr match: 529
Proportion of records with 1x obj match and corr match: 0.26893746822572445 

Records with 1x obj match but no corr match: 361
Proportion of records with 1x obj match but no corr match: 0.1835282155566853 

Records with no obj match: 725
Proportion of records with no obj match: 0.36858159633960347 



In [33]:
uic_dates_df.to_csv(r'D:\CPOdata\2017_joined_cc_MADW_dates_df.csv', encoding='utf-8', index=False)